# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} x_1 ~~ s.t. ~ x_1 + x_2 + x_3 = 1 ~, ~~ x \ge 0$

Dual: $\underset{\lambda \in \mathbb{R}, s \in \mathbb{R^3}}{\max} \lambda ~~ s.t. ~ [1~1~1]' \lambda + s = [1~0~0]'~, ~~s\ge0$

### 2)

In [98]:
using LinearAlgebra
using LaTeXStrings
using DataStructures
using BenchmarkTools

In [44]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end


display_results (generic function with 1 method)

In [45]:
function Compute_AlphaAff(alpha_step, v_k, dir_v_k)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 1 && all(v_k + (alpha+alpha_step) * dir_v_k .>= 0) 
        alpha += alpha_step
    end
    return alpha
end

function Compute_AlphaMax(alpha_step, v_k, dir_v_k)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 100/99 && all(v_k + (alpha+alpha_step) * dir_v_k .>= 0) 
        alpha += alpha_step
    end
    return alpha
end




Compute_AlphaMax (generic function with 1 method)

In [101]:

function MehrotraPC1(c, A, b, init_vect, max_iter, eps = 1e-10, alpha_step=1e-2, display=true)

    x_0, lambda_0, s_0 = init_vect
    x_k, lambda_k, s_k = copy(x_0), copy(lambda_0), copy(s_0)
    n_rows, n_cols = size(A) 
    
    rb_0 = A * x_0 - b
    rc_0 = A' * lambda_0 + s_0 - c
    mu_0 = s_0' * x_0 / n_cols
    Xk = Diagonal(x_k)
    Sk = Diagonal(s_k)
    mu_k = s_k' * x_k / n_cols
    rb_k = A * x_k - b
    rc_k = A' * lambda_k + s_k - c
    
    k = 0
    pdd, cond_rb, cond_rc = 1, 1, 1
    e = ones(n_cols, 1)
    n_c = norm(c)
    n_b = norm(b)
    
    while k <= max_iter && pdd > eps && cond_rb > eps && cond_rc > eps  # 3 epsilon différents en pratique???

        Jacob_Fk = [zeros(n_cols, n_cols) A' I(n_cols)
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk_aff = [-rc_k
                  -rb_k
                  -Xk*Sk*e]
        

        dir_aff_k = Jacob_Fk\Fk_aff
        

        alpha_aff_pri = Compute_AlphaAff(alpha_step, x_k, dir_aff_k[1:n_cols])
        alpha_aff_dual = Compute_AlphaAff(alpha_step, s_k, dir_aff_k[n_rows+n_cols+1: end])
        mu_aff = (x_k + alpha_aff_pri * dir_aff_k[1:n_cols])' * 
                    (s_k + alpha_aff_dual * dir_aff_k[n_rows+n_cols+1: end]) / n_cols
        sigma = (mu_aff / mu_k)^3
     
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e - Diagonal(dir_aff_k[1:n_cols])*Diagonal(dir_aff_k[n_rows+n_cols+1: end])*e] 
        dir_cc_k = Jacob_Fk\Fk_cc
         
        dir_k = dir_aff_k .+ dir_cc_k
         
        alpha_max_pri = Compute_AlphaMax(alpha_step, x_k, dir_k[1:n_cols])
        alpha_max_dual = Compute_AlphaMax(alpha_step, s_k, dir_k[n_rows+n_cols+1: end])
     

        alpha_k_pri = min(0.99*alpha_max_pri, 1)
        alpha_k_dual = min(0.99*alpha_max_dual, 1)
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_k_dual * dir_k[n_cols+1: n_rows+n_cols]
        s_k += alpha_k_dual * dir_k[n_rows+n_cols+1: end]
        
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        mu_k = s_k' * x_k / n_cols
        rb_k = A * x_k - b
        rc_k = A' * lambda_k + s_k - c
        
        # valeurs pour les conditions d'arret:
        pdd = abs(c' * x_k - b' * lambda_k) / (1 + abs(c' * x_k))
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        
        k += 1
        
        # affichage
        if display == true
            println("---------------------------------------------------------------")
            println("Iter ", k, ":   pdd = ", pdd, " |  cond_rb = ", cond_rb, " |  cond_rc = ", cond_rc)
        end
    end
    
    if display == true
        criteria = [k > max_iter,  pdd <= eps, cond_rb <= eps, cond_rc <= eps]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

MehrotraPC1 (generic function with 4 methods)

In [102]:
### probleme 1
c = [1, 0 ,0]
A = Matrix([1, 1, 1]')
b = [1];

In [103]:
# initialisation 1 
x01 = [4, 1, 3]
lambda01 = [-2]
s01 = [3, 4, 2]
init_vect1 = (x01, lambda01, s01);
res_mpc1 =  MehrotraPC1(c, A, b, init_vect1, 20);
display_results(res_mpc1) 

---------------------------------------------------------------
Iter 1:   pdd = 1.4765522896158283 |  cond_rb = 0.00034999999999763 |  cond_rc = 9.999999999943388e-5
---------------------------------------------------------------
Iter 2:   pdd = 0.1401237627258906 |  cond_rb = 3.500000000933312e-8 |  cond_rc = 8.919999999967843e-6
---------------------------------------------------------------
Iter 3:   pdd = 0.0027960307319132233 |  cond_rb = 6.964999688108264e-10 |  cond_rc = 8.919999999051909e-10
---------------------------------------------------------------
Iter 4:   pdd = 5.159304917933026e-5 |  cond_rb = 1.3860357306327842e-11 |  cond_rc = 8.919999992546696e-12

 stopping criterion = ["cond_rb <= eps", "cond_rc <= eps"]

-----------------------------------------------------------------------
------------------------------- RESULTS -------------------------------


OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [4.73687e-5, 0.499572, 0.500381]
  "lambda_opt" => [-4.22679e-6]
  "s_opt"      => [1.0, 4.22681e-6, 4.22679e-6]
  "n_iter"     => 4
  "pdd"        => 5.1593e-5
  "cond_rb"    => 1.38604e-11
  "cond_rc"    => 8.92e-12

In [106]:
@benchmark MehrotraPC1(c, A, b, init_vect1, 20, 1e-10, 1e-2, false)

BenchmarkTools.Trial: 
  memory estimate:  573.03 KiB
  allocs estimate:  6653
  --------------
  minimum time:     338.399 μs (0.00% GC)
  median time:      478.500 μs (0.00% GC)
  mean time:        561.346 μs (14.37% GC)
  maximum time:     9.156 ms (91.55% GC)
  --------------
  samples:          8835
  evals/sample:     1